**Table of contents**<a id='toc0_'></a>    
- 1. [Advanced: Middle-square method for generating random numbers](#toc1_)    
- 2. [Advanced: Gauss-Hermite quadrature](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

Random numbers advanced
============


The notebook contains two advanced sections

Section 1 is a method showing how you can generate a pseudo-random number without any packages. This is mostly to show you the intution behind random numbers on a computer. In practice, it is more efficient to just use a package like Numpy's Random.

Section 2 is a method showing how you can simulate a distribution in a more efficient way than just drawing random numbers from the distribution like Monte Carlo. 

In [1]:
import math
import numpy as np

# 1. <a id='toc1_'></a>[Advanced: Middle-square method for generating random numbers](#toc0_)

Proposed by **John von Neumann**:

1. Start with a $N$ digit number
2. Square the number
3. Pad the number with leading zeros making it a $2N$ digit number
4. Extract the middle $N$ digits (*your random number*)
5. Return to step 1 to generate one more

> **Pro:** Simple and easy to implement. Conceptually somewhat similar to more advanced methods (e.g. *Mersenne-Twister* used by *numpy*).
>
> **Con:** Cycles can be no longer than $8^N$ periods. Many repeating cycles are very short. Internal state is directly observable.
>
> **Conclusion:** Can not be used in practice.

**Code:** An implementation in Python for $N = 4$ digit random integers:

In [2]:
def rng(number,max_iter=100): 
    
    already_seen = [] # list of seen numbers
    
    i = 0
    while number not in already_seen and i < max_iter:
        
        already_seen.append(number)
        squared = number**2
        padded = str(squared).zfill(8) # add leading zeros
        number = int(padded[2:6]) # extract middle 4 numbers
        
        print(f"square = {squared:8d}, padded = {padded} -> {number:4d}")
        i += 1

A reasonable cycle:

In [3]:
rng(4653)

square = 21650409, padded = 21650409 -> 6504
square = 42302016, padded = 42302016 -> 3020
square =  9120400, padded = 09120400 -> 1204
square =  1449616, padded = 01449616 -> 4496
square = 20214016, padded = 20214016 -> 2140
square =  4579600, padded = 04579600 -> 5796
square = 33593616, padded = 33593616 -> 5936
square = 35236096, padded = 35236096 -> 2360
square =  5569600, padded = 05569600 -> 5696
square = 32444416, padded = 32444416 -> 4444
square = 19749136, padded = 19749136 -> 7491
square = 56115081, padded = 56115081 -> 1150
square =  1322500, padded = 01322500 -> 3225
square = 10400625, padded = 10400625 -> 4006
square = 16048036, padded = 16048036 ->  480
square =   230400, padded = 00230400 -> 2304
square =  5308416, padded = 05308416 -> 3084
square =  9511056, padded = 09511056 -> 5110
square = 26112100, padded = 26112100 -> 1121
square =  1256641, padded = 01256641 -> 2566
square =  6584356, padded = 06584356 -> 5843
square = 34140649, padded = 34140649 -> 1406
square =  

A short cycle:

In [4]:
rng(540)

square =   291600, padded = 00291600 -> 2916
square =  8503056, padded = 08503056 -> 5030
square = 25300900, padded = 25300900 -> 3009
square =  9054081, padded = 09054081 ->  540


No cycle at all:

In [5]:
rng(3792)

square = 14379264, padded = 14379264 -> 3792


# 2. <a id='toc2_'></a>[Advanced: Gauss-Hermite quadrature](#toc0_)

**Problem:** Numerical integration by Monte Carlo is **slow**.

**Solution:** Use smarter integration formulas on the form

$$
\mathbb{E}[g(x)] \approx \sum_{i=1}^{n} w_ig(x_i) 
$$

where $(x_i,w_i), \forall n \in \{1,2,\dots,N\}$, are called **quadrature nodes and weights** and are provided by some theoretical formula depending on the distribution of $x$.

**Example I, Normal:** If $x \sim \mathcal{N}(\mu,\sigma)$ and $g(x_i) = (x_i-2)^2$ then we can use [Gauss-Hermite quadrature](https://en.wikipedia.org/wiki/Gauss%E2%80%93Hermite_quadrature) as implemented below.

In [6]:
def gauss_hermite(n):
    """ gauss-hermite nodes

    Args:

        n (int): number of points

    Returns:

        x (numpy.ndarray): nodes of length n
        w (numpy.ndarray): weights of length n

    """

    # a. calculations
    i = np.arange(1,n)
    a = np.sqrt(i/2)
    CM = np.diag(a,1) + np.diag(a,-1)
    L,V = np.linalg.eig(CM)
    I = L.argsort()
    V = V[:,I].T

    # b. nodes and weights
    x = L[I]
    w = np.sqrt(math.pi)*V[:,0]**2

    return x,w

In [7]:
def normal_gauss_hermite(sigma, n=7, mu=None, exp=False):
    """ normal gauss-hermite nodes

    Args:

        sigma (double): standard deviation
        n (int): number of points
        mu (double,optinal): mean
        exp (bool,optinal): take exp and correct mean (if not specified)

    Returns:

        x (numpy.ndarray): nodes of length n
        w (numpy.ndarray): weights of length n

    """

    if sigma == 0.0 or n == 1:
        x = np.ones(n)
        if mu is not None:
            x += mu
        w = np.ones(n)
        return x,w

    # a. GaussHermite
    x,w = gauss_hermite(n)
    x *= np.sqrt(2)*sigma 

    # b. log-normality
    if exp:
        if mu is None:
            x = np.exp(x - 0.5*sigma**2)
        else:
            x = np.exp(x + mu)
    else:
        if mu is None:
            x = x 
        else:
            x = x + mu

    w /= np.sqrt(math.pi)

    return x,w

**Results:** Becuase the function is "nice", very few quadrature points are actually needed (*not generally true*).

In [8]:
mu = 0.1
sigma = 0.5
def g(x):
    return (x-2)**2

In [9]:
for n in [1,2,3,5,7,9,11]:
    x,w = normal_gauss_hermite(mu=mu,sigma=sigma,n=n)
    result = np.sum(w*g(x))
    print(f'n = {n:3d}: {result:.10f}')

n =   1: 0.8100000000
n =   2: 3.8600000000
n =   3: 3.8600000000
n =   5: 3.8600000000
n =   7: 3.8600000000
n =   9: 3.8600000000
n =  11: 3.8600000000


**Example II, log-normal ([more info](https://en.wikipedia.org/wiki/Log-normal_distribution)):** 

1. Let $\log x \sim \mathcal{N}(\mu,\sigma)$. 
2. Gauss-Hermite quadrature nodes and weights can be used with the option `exp=True`. 
3. To ensure $\mathbb{E}[x] = 1$ then $\mu = -0.5\sigma^2$.

In [10]:
z = np.random.normal(size=1_000_000,scale=sigma)

print('mean(x) when mu = 0')
x,w = normal_gauss_hermite(mu=0,sigma=sigma,n=7,exp=True)
print(f'MC: {np.mean(np.exp(z)):.4f}')
print(f'Gauss-Hermite: {np.sum(x*w):.4f}')
print('')

print('mean(x), mu = -0.5*sigma^2')
x,w = normal_gauss_hermite(sigma=sigma,n=7,exp=True)
print(f'MC: {np.mean(np.exp(z-0.5*sigma**2)):.4f}')
print(f'Gauss-Hermite: {np.sum(x*w):.4f}')

mean(x) when mu = 0
MC: 1.1336
Gauss-Hermite: 1.1331

mean(x), mu = -0.5*sigma^2
MC: 1.0004
Gauss-Hermite: 1.0000
